In [4]:
# spark.stop()

In [2]:
import os
from pyspark.sql import SparkSession, types as t, functions as F

In [3]:
# https://storage.googleapis.com/hadoop-lib/gcs/gcs-connector-hadoop3-2.2.14.jar

spark = (
    SparkSession
    .builder
    # .master("spark://spark-master:7077")
    .appName("Testing Transformations")
    .config("spark.jars", "https://storage.googleapis.com/hadoop-lib/gcs/gcs-connector-hadoop3-2.2.14.jar") # GCS Connector
    .getOrCreate()
)

# Google Cloud Service Account Credentials
spark._jsc.hadoopConfiguration().set("google.cloud.auth.service.account.json.keyfile",os.environ.get("GOOGLE_APPLICATION_CREDENTIALS"))

spark

25/04/28 10:22:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [ ]:
drug_schema = t.StructType(
    [
        t.StructField('patientid',t.StringType(),True),
        t.StructField('medicinalproduct',t.StringType(),True),
        t.StructField('activesubstancename',t.StringType(),True),
        t.StructField('drugadministrationroute',t.StringType(),True),
        t.StructField('drugstartdate',t.StringType(),True),
        t.StructField('drugenddate',t.StringType(),True),
        t.StructField('drugdosagetext',t.StringType(),True),
        t.StructField('drugstructuredosagenumb',t.StringType(),True),
        t.StructField('drugstructuredosageunit',t.StringType(),True),
        t.StructField('drugtreatmentduration',t.StringType(),True),
        t.StructField('drugtreatmentdurationunit',t.StringType(),True),
        t.StructField('drugrecurreadministration',t.StringType(),True),
    ]
)

reaction_schema = t.StructType(
    [
        t.StructField('patientid',t.StringType(),True),
        t.StructField('reactionmeddrapt',t.StringType(),True),
        t.StructField('reactionoutcome',t.StringType(),True),
    ]
)

In [3]:
bucket='gs://zoomcamp-454219-ade-pipeline/processed/pq/'

# Patient

### Load patient

In [4]:
patient_schema = t.StructType(
    [
        t.StructField('patientid', t.StringType(),True),
        t.StructField('patientagegroup', t.StringType(),True),
        t.StructField('patientonsetage', t.StringType(),True),
        t.StructField('patientonsetageunit', t.StringType(),True),
        t.StructField('patientsex', t.StringType(),True),
        t.StructField('patientweight', t.StringType(),True),
        t.StructField('serious', t.IntegerType(), True),
        t.StructField('seriousnessdeath', t.IntegerType(), True),
        t.StructField('seriousnesshospitalization', t.IntegerType(), True),
        t.StructField('seriousnessdisabling', t.IntegerType(), True),
        t.StructField('seriousnesslifethreatening', t.IntegerType(), True),
        t.StructField('seriousnessother', t.IntegerType(), True),
        t.StructField('receivedate', t.IntegerType(), True),
        t.StructField('receiptdate', t.IntegerType(), True),
        t.StructField('safetyreportid', t.IntegerType(), True)
    ]
)

patient = (
    spark
    .read
    .schema(patient_schema)
    .parquet(bucket+'patient/*/*')
    )
print(f"Count: {patient.count()}")
print(patient.printSchema())
patient.show()

Py4JJavaError: An error occurred while calling o41.parquet.
: java.io.IOException: No FileSystem for scheme: gs
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:2660)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:2667)
	at org.apache.hadoop.fs.FileSystem.access$200(FileSystem.java:94)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:2703)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:2685)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:373)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:295)
	at org.apache.spark.sql.execution.streaming.FileStreamSink$.hasMetadata(FileStreamSink.scala:46)
	at org.apache.spark.sql.execution.datasources.DataSource.resolveRelation(DataSource.scala:361)
	at org.apache.spark.sql.DataFrameReader.loadV1Source(DataFrameReader.scala:279)
	at org.apache.spark.sql.DataFrameReader.$anonfun$load$2(DataFrameReader.scala:268)
	at scala.Option.getOrElse(Option.scala:189)
	

In [59]:
patient.select("patientagegroup").distinct().show()


25/04/27 10:01:30 ERROR Executor: Exception in task 3.0 in stage 84.0 (TID 2637)
org.apache.spark.sql.execution.QueryExecutionException: Parquet column cannot be converted in file gs://zoomcamp-454219-ade-pipeline/data/pq/patient/2004/drug-event-part-8-of-20.parquet. Column: [patientagegroup], Expected: string, Found: INT32
	at org.apache.spark.sql.errors.QueryExecutionErrors$.unsupportedSchemaColumnConvertError(QueryExecutionErrors.scala:724)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:278)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:116)
	at org.apache.spark.sql.execution.FileSourceScanExec$$anon$1.hasNext(DataSourceScanExec.scala:554)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.columnartorow_nextBatch_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.hashAgg_doAggregat

Py4JJavaError: An error occurred while calling o1059.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 84.0 failed 1 times, most recent failure: Lost task 3.0 in stage 84.0 (TID 2637) (d8fd851c15c0 executor driver): org.apache.spark.sql.execution.QueryExecutionException: Parquet column cannot be converted in file gs://zoomcamp-454219-ade-pipeline/data/pq/patient/2004/drug-event-part-8-of-20.parquet. Column: [patientagegroup], Expected: string, Found: INT32
	at org.apache.spark.sql.errors.QueryExecutionErrors$.unsupportedSchemaColumnConvertError(QueryExecutionErrors.scala:724)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:278)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:116)
	at org.apache.spark.sql.execution.FileSourceScanExec$$anon$1.hasNext(DataSourceScanExec.scala:554)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$Generate

25/04/27 10:01:30 WARN TaskSetManager: Lost task 12.0 in stage 84.0 (TID 2646) (d8fd851c15c0 executor driver): TaskKilled (Stage cancelled)


25/04/27 10:01:30 WARN TaskSetManager: Lost task 7.0 in stage 84.0 (TID 2641) (d8fd851c15c0 executor driver): TaskKilled (Stage cancelled)
25/04/27 10:01:30 WARN TaskSetManager: Lost task 9.0 in stage 84.0 (TID 2643) (d8fd851c15c0 executor driver): TaskKilled (Stage cancelled)
25/04/27 10:01:30 WARN TaskSetManager: Lost task 4.0 in stage 84.0 (TID 2638) (d8fd851c15c0 executor driver): TaskKilled (Stage cancelled)
25/04/27 10:01:30 WARN TaskSetManager: Lost task 16.0 in stage 84.0 (TID 2650) (d8fd851c15c0 executor driver): TaskKilled (Stage cancelled)
25/04/27 10:01:30 WARN TaskSetManager: Lost task 6.0 in stage 84.0 (TID 2640) (d8fd851c15c0 executor driver): TaskKilled (Stage cancelled)
25/04/27 10:01:30 WARN TaskSetManager: Lost task 19.0 in stage 84.0 (TID 2653) (d8fd851c15c0 executor driver): TaskKilled (Stage cancelled)
25/04/27 10:01:30 WARN TaskSetManager: Lost task 15.0 in stage 84.0 (TID 2649) (d8fd851c15c0 executor driver): TaskKilled (Stage cancelled)
25/04/27 10:01:30 WARN T

25/04/27 10:01:30 WARN TaskSetManager: Lost task 13.0 in stage 84.0 (TID 2647) (d8fd851c15c0 executor driver): TaskKilled (Stage cancelled)
25/04/27 10:01:30 WARN TaskSetManager: Lost task 5.0 in stage 84.0 (TID 2639) (d8fd851c15c0 executor driver): TaskKilled (Stage cancelled)


### Perform Cast

In [52]:
# Cast
patient = (
    patient
    .withColumn("patientonsetage", F.col('patientonsetage').cast(t.IntegerType()))
    .withColumn("patientonsetageunit", F.col('patientonsetageunit').cast(t.IntegerType()))
    .withColumn("patientsex", F.col('patientsex').cast(t.IntegerType()))
    # .withColumn("patientweight", F.col('patientweight').cast(t.FloatType()))
)

### Perform Transformation

In [53]:
patient.printSchema()

root
 |-- patientid: string (nullable = true)
 |-- patientagegroup: string (nullable = true)
 |-- patientonsetage: integer (nullable = true)
 |-- patientonsetageunit: integer (nullable = true)
 |-- patientsex: integer (nullable = true)
 |-- patientweight: string (nullable = true)
 |-- serious: integer (nullable = true)
 |-- seriousnessdeath: integer (nullable = true)
 |-- seriousnesshospitalization: integer (nullable = true)
 |-- seriousnessdisabling: integer (nullable = true)
 |-- seriousnesslifethreatening: integer (nullable = true)
 |-- seriousnessother: integer (nullable = true)
 |-- receivedate: integer (nullable = true)
 |-- receiptdate: integer (nullable = true)
 |-- safetyreportid: integer (nullable = true)



In [54]:
# Patient Column transformations

patient = patient.withColumn(
    "patientagegroup",
    (
        F
        .when(F.col("patientagegroup") == '1', "Neonate")
        .when(F.col("patientagegroup") == '2', "Infant")
        .when(F.col("patientagegroup") == '3', "Child")
        .when(F.col("patientagegroup") == '4', "Adolescent")
        .when(F.col("patientagegroup") == '5', "Adult")
        .when(F.col("patientagegroup") == '6', "Elderly")
        .otherwise(None)
     ).cast(t.StringType())
)

# Normalize patientage
patient = patient.withColumn(
    "patientage",
    (
        F
        .when(F.col("patientonsetageunit") == 800, F.col("patientonsetage") * 10)
        .when(F.col("patientonsetageunit") == 801, F.col("patientonsetage") * 1)
        .when(F.col("patientonsetageunit") == 802, F.col("patientonsetage") / 12)
        .when(F.col("patientonsetageunit") == 803, F.col("patientonsetage") / 52.143)
        .when(F.col("patientonsetageunit") == 804, F.col("patientonsetage") / 365.25)
        .when(F.col("patientonsetageunit") == 805, F.col("patientonsetage") / 8766)
        .otherwise(None)
    ).cast(t.FloatType())
).drop("patientonsetageunit", "patientonsetage")

patient = patient.withColumn(
    "patientsex",
    (
        F
        .when(F.col("patientsex") == 1, "Male")
        .when(F.col("patientsex") == 2, "Female")
        .otherwise(None)
    ).cast(t.StringType())
)

# patient = patient.withColumn(
#     "patientweight",
#     (
#         F
#         .when(
#             F.col("patientweight").rlike(r"^\d+(\.\d+)?$"),
#             F.col("patientweight").cast(t.FloatType()))
#         .otherwise(None)
#      )
# )

# patient = patient.withColumn(
#     "serious",
#     (
#         F
#         .when(F.col("serious") == 1, True)
#         .when(F.col("serious") == 2, False)
#         .otherwise(None)
#      )
# )

# patient = patient.withColumn(
#     "seriousnessdeath",
#     (
#         F
#         .when(F.col("seriousnessdeath") == 1, True)
#         .otherwise(False)
#      )
# )

# patient = patient.withColumn(
#     "seriousnesshospitalization",
#     (
#         F
#         .when(F.col("seriousnesshospitalization") == 1, True)
#         .otherwise(False)
#      )
# )

# patient = patient.withColumn(
#     "seriousnessdisabling",
#     (
#         F
#         .when(F.col("seriousnessdisabling") == 1, True)
#         .otherwise(False)
#      )
# )

# patient = patient.withColumn(
#     "seriousnesslifethreatening",
#     (
#         F
#         .when(F.col("seriousnesslifethreatening") == 1, True)
#         .otherwise(False)
#      )
# )

# patient = patient.withColumn(
#     "seriousnessother",
#     (
#         F
#         .when(F.col("seriousnessother") == 1, True)
#         .otherwise(False)
#      )
# )

In [55]:
patient.printSchema()

root
 |-- patientid: string (nullable = true)
 |-- patientagegroup: string (nullable = true)
 |-- patientsex: string (nullable = true)
 |-- patientweight: string (nullable = true)
 |-- serious: integer (nullable = true)
 |-- seriousnessdeath: integer (nullable = true)
 |-- seriousnesshospitalization: integer (nullable = true)
 |-- seriousnessdisabling: integer (nullable = true)
 |-- seriousnesslifethreatening: integer (nullable = true)
 |-- seriousnessother: integer (nullable = true)
 |-- receivedate: integer (nullable = true)
 |-- receiptdate: integer (nullable = true)
 |-- safetyreportid: integer (nullable = true)
 |-- patientage: float (nullable = true)



### Display rows

In [56]:
patient.show()

+--------------------+---------------+----------+-------------+-------+----------------+--------------------------+--------------------+--------------------------+----------------+-----------+-----------+--------------+----------+
|           patientid|patientagegroup|patientsex|patientweight|serious|seriousnessdeath|seriousnesshospitalization|seriousnessdisabling|seriousnesslifethreatening|seriousnessother|receivedate|receiptdate|safetyreportid|patientage|
+--------------------+---------------+----------+-------------+-------+----------------+--------------------------+--------------------+--------------------------+----------------+-----------+-----------+--------------+----------+
|c03ef5e8-17e8-4b0...|           null|    Female|         null|   null|            null|                      null|                null|                      null|            null|       null|       null|          null|      null|
|be397b97-b8c8-42a...|           null|      Male|         null|   null|     

In [57]:
patient.groupBy("patientagegroup").count().show()

25/04/27 10:01:00 ERROR Executor: Exception in task 2.0 in stage 78.0 (TID 2453)
org.apache.spark.sql.execution.QueryExecutionException: Parquet column cannot be converted in file gs://zoomcamp-454219-ade-pipeline/data/pq/patient/2004/drug-event-part-19-of-20.parquet. Column: [patientagegroup], Expected: string, Found: INT32
	at org.apache.spark.sql.errors.QueryExecutionErrors$.unsupportedSchemaColumnConvertError(QueryExecutionErrors.scala:724)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:278)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:116)
	at org.apache.spark.sql.execution.FileSourceScanExec$$anon$1.hasNext(DataSourceScanExec.scala:554)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.columnartorow_nextBatch_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.hashAgg_doAggrega

Py4JJavaError: An error occurred while calling o1038.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 78.0 failed 1 times, most recent failure: Lost task 2.0 in stage 78.0 (TID 2453) (d8fd851c15c0 executor driver): org.apache.spark.sql.execution.QueryExecutionException: Parquet column cannot be converted in file gs://zoomcamp-454219-ade-pipeline/data/pq/patient/2004/drug-event-part-19-of-20.parquet. Column: [patientagegroup], Expected: string, Found: INT32
	at org.apache.spark.sql.errors.QueryExecutionErrors$.unsupportedSchemaColumnConvertError(QueryExecutionErrors.scala:724)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.nextIterator(FileScanRDD.scala:278)
	at org.apache.spark.sql.execution.datasources.FileScanRDD$$anon$1.hasNext(FileScanRDD.scala:116)
	at org.apache.spark.sql.execution.FileSourceScanExec$$anon$1.hasNext(DataSourceScanExec.scala:554)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$Generat

25/04/27 10:01:00 WARN TaskSetManager: Lost task 5.0 in stage 78.0 (TID 2456) (d8fd851c15c0 executor driver): TaskKilled (Stage cancelled)


25/04/27 10:01:00 WARN TaskSetManager: Lost task 12.0 in stage 78.0 (TID 2463) (d8fd851c15c0 executor driver): TaskKilled (Stage cancelled)
25/04/27 10:01:00 WARN TaskSetManager: Lost task 18.0 in stage 78.0 (TID 2469) (d8fd851c15c0 executor driver): TaskKilled (Stage cancelled)
25/04/27 10:01:00 WARN TaskSetManager: Lost task 19.0 in stage 78.0 (TID 2470) (d8fd851c15c0 executor driver): TaskKilled (Stage cancelled)
25/04/27 10:01:01 WARN TaskSetManager: Lost task 13.0 in stage 78.0 (TID 2464) (d8fd851c15c0 executor driver): TaskKilled (Stage cancelled)
25/04/27 10:01:01 WARN TaskSetManager: Lost task 8.0 in stage 78.0 (TID 2459) (d8fd851c15c0 executor driver): TaskKilled (Stage cancelled)
25/04/27 10:01:01 WARN TaskSetManager: Lost task 16.0 in stage 78.0 (TID 2467) (d8fd851c15c0 executor driver): TaskKilled (Stage cancelled)


25/04/27 10:01:01 WARN TaskSetManager: Lost task 14.0 in stage 78.0 (TID 2465) (d8fd851c15c0 executor driver): TaskKilled (Stage cancelled)


# Drug

### Load Drug

In [11]:
drug = (
    spark
    .read
    .parquet(bucket+'drug/*/*')
    )

drug.printSchema()

root
 |-- patientid: string (nullable = true)
 |-- medicinalproduct: string (nullable = true)
 |-- activesubstancename: string (nullable = true)
 |-- drugindication: string (nullable = true)
 |-- drugadministrationroute: string (nullable = true)
 |-- drugstartdate: string (nullable = true)
 |-- drugenddate: string (nullable = true)
 |-- drugdosagetext: string (nullable = true)
 |-- drugstructuredosagenumb: string (nullable = true)
 |-- drugstructuredosageunit: string (nullable = true)
 |-- drugtreatmentduration: string (nullable = true)
 |-- drugtreatmentdurationunit: string (nullable = true)
 |-- drugrecurreadministration: string (nullable = true)



### Perform Cast

In [12]:
# Cast
drug = (
    drug
    .withColumn("drugstructuredosageunit", F.col("drugstructuredosageunit").cast(t.StringType()))
    .withColumn("drugrecurreadministration", F.col("drugrecurreadministration").cast(t.IntegerType()))
)

### Perform Transformation

In [13]:
route_mapping = {
    "001": "Auricular (otic)",
    "002": "Buccal",
    "003": "Cutaneous",
    "004": "Dental",
    "005": "Endocervical",
    "006": "Endosinusial",
    "007": "Endotracheal",
    "008": "Epidural",
    "009": "Extra-amniotic",
    "010": "Hemodialysis",
    "011": "Intra corpus cavernosum",
    "012": "Intra-amniotic",
    "013": "Intra-arterial",
    "014": "Intra-articular",
    "015": "Intra-uterine",
    "016": "Intracardiac",
    "017": "Intracavernous",
    "018": "Intracerebral",
    "019": "Intracervical",
    "020": "Intracisternal",
    "021": "Intracorneal",
    "022": "Intracoronary",
    "023": "Intradermal",
    "024": "Intradiscal (intraspinal)",
    "025": "Intrahepatic",
    "026": "Intralesional",
    "027": "Intralymphatic",
    "028": "Intramedullar (bone marrow)",
    "029": "Intrameningeal",
    "030": "Intramuscular",
    "031": "Intraocular",
    "032": "Intrapericardial",
    "033": "Intraperitoneal",
    "034": "Intrapleural",
    "035": "Intrasynovial",
    "036": "Intratumor",
    "037": "Intrathecal",
    "038": "Intrathoracic",
    "039": "Intratracheal",
    "040": "Intravenous bolus",
    "041": "Intravenous drip",
    "042": "Intravenous (not otherwise specified)",
    "043": "Intravesical",
    "044": "Iontophoresis",
    "045": "Nasal",
    "046": "Occlusive dressing technique",
    "047": "Ophthalmic",
    "048": "Oral",
    "049": "Oropharingeal",
    "050": "Other",
    "051": "Parenteral",
    "052": "Periarticular",
    "053": "Perineural",
    "054": "Rectal",
    "055": "Respiratory (inhalation)",
    "056": "Retrobulbar",
    "057": "Sunconjunctival",
    "058": "Subcutaneous",
    "059": "Subdermal",
    "060": "Sublingual",
    "061": "Topical",
    "062": "Transdermal",
    "063": "Transmammary",
    "064": "Transplacental",
    "065": "Unknown",
    "066": "Urethral",
    "067": "Vaginal"
}

# Fix missing parts of the date
drug = (
    drug
    .withColumn(
        "drugstartdate",
        (
            F
            .when(F.length("drugstartdate") == 4, F.concat("drugstartdate",F.lit("0101")))
            .when(F.length("drugstartdate") == 6, F.concat("drugstartdate",F.lit("01")))
            .otherwise(F.col("drugstartdate"))
        )
    )
)

drug = (
    drug
    .withColumn(
        "drugstartdate",
        (
            F.to_date("drugstartdate","yyyyMMdd")
        )
    )
)

drug = (
    drug
    .withColumn(
        "drugenddate",
        (
            F
            .when(F.length("drugenddate") == 4, F.concat("drugenddate",F.lit("0101")))
            .when(F.length("drugenddate") == 6, F.concat("drugenddate",F.lit("01")))
            .otherwise(F.col("drugenddate"))
        )
    )
)

drug = (
    drug
    .withColumn(
        "drugenddate",
        (
            F.to_date("drugenddate","yyyyMMdd")
        )
    )
)

map_expr = F.create_map([F.lit(i) for i in sum(route_mapping.items(),())])

drug = (
    drug
    .withColumn(
        "drugadministrationroute",
        map_expr[F.col("drugadministrationroute")]
    )
)

In [14]:
drug.show()

+--------------------+--------------------+--------------------+--------------------+-----------------------+-------------+-----------+--------------------+-----------------------+-----------------------+---------------------+-------------------------+-------------------------+
|           patientid|    medicinalproduct| activesubstancename|      drugindication|drugadministrationroute|drugstartdate|drugenddate|      drugdosagetext|drugstructuredosagenumb|drugstructuredosageunit|drugtreatmentduration|drugtreatmentdurationunit|drugrecurreadministration|
+--------------------+--------------------+--------------------+--------------------+-----------------------+-------------+-----------+--------------------+-----------------------+-----------------------+---------------------+-------------------------+-------------------------+
|070ec3d6-9b34-412...|            ENTRESTO|SACUBITRIL\VALSARTAN|                null|                   Oral|         null|       null|50 MG, BID (24/26...|       

# Reaction

### Load Reaction

In [15]:
reaction = (
    spark
    .read
    .parquet(bucket+'reaction/*/*')
)

reaction.printSchema()

root
 |-- patientid: string (nullable = true)
 |-- reactionmeddrapt: string (nullable = true)
 |-- reactionoutcome: string (nullable = true)



### Perform Cast

In [16]:
# Cast
reaction = (
    reaction
    .withColumn("reactionoutcome", F.col("reactionoutcome").cast(t.IntegerType()))
)
reaction.printSchema()

root
 |-- patientid: string (nullable = true)
 |-- reactionmeddrapt: string (nullable = true)
 |-- reactionoutcome: integer (nullable = true)

